In [1]:
import numpy as np

In [118]:
n = 10
p = 3
N = np.random.rand(n)
b = np.random.rand(n,n)
S = np.random.rand(n)
k = np.random.rand(p)
delta = 1e-8

# Breakage function

## Discretized breakage birth
\begin{equation}
R_i^{[1]}=\sum_{j=i}^{n}b_{i,j}S_jN_j
\end{equation}
where $S_i$ is the selection rate for interval $i$ and $b_{i,j}$ is the number of fragments from $j$ to $i$ which occurs in $1\sim n$
## Discretized breakage death
\begin{equation}
R_i^{[2]}=S_iN_i
\end{equation}
which occurs in $2\sim n$

In [3]:
def breakage(N, bmat, Svec):
    n = len(N)
    R1 = np.zeros(n)
    
    for i in range(n):
        R1[i] = np.sum(bmat[i, i:] * Svec[i:] * N[i:])
        
    R2 = Svec[1:] * N[1:]
    R2 = np.insert(R2, 0, 0.0)
        
    dNdt = R1 - R2

    return dNdt

In [87]:
NN = np.repeat(N.reshape(1,n),n,axis=0)

In [88]:
def l():
    for i in range(n):
        breakage(NN[i], b, S)
    
def l2():
    for n in NN:
        breakage(n, b, S)

In [89]:
%timeit l
%timeit l2

19.7 ns ± 0.105 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
18.4 ns ± 0.0342 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [4]:
%timeit breakage(N, b, S)

87.5 µs ± 734 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Phi

## Calculation of Jacobian in ODE models
In ODE models, the sensitivity matrix cannot be obtained by a simple differentiation. Instead, we can get differential equation for $\mathbf{J}$.
Differentiate both side of $ \frac{\text{d}\mathbf{y}}{\text{d}t}=\mathbf{f}(\mathbf{y}(t),\mathbf{k})$ and apply the chain rule,
\begin{align}
\frac{\partial}{\partial\mathbf{k}}\left(\frac{\text{d}\mathbf{y}}{\text{d}t}\right)=\frac{\text{d}}{\text{d}t}\left(\frac{\partial\mathbf{y}}{\partial\mathbf{k}}\right)&=\frac{\partial}{\partial\mathbf{k}}\mathbf{f}(\mathbf{y}(t),\mathbf{k})\\
&=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\frac{\partial\mathbf{y}}{\partial\mathbf{k}}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}\frac{\partial\mathbf{k}}{\partial\mathbf{k}}\\
&=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\frac{\partial\mathbf{y}}{\partial\mathbf{k}}+\frac{\partial\mathbf{f}}{\partial\mathbf{k}}
\end{align}
Hence,
\begin{equation*}
\frac{\text{d}\mathbf{J}(t)}{\text{d}t}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{J}(t)+\frac{\partial\mathbf{f}}{\partial\mathbf{k}};\hspace{10mm}\mathbf{J}(t_0)=0
\end{equation*}


## Contruction of ODE system with Jacobian
The Jacobian or the sensitivity matrix is
\begin{equation}
\mathbf{J}(t)=\frac{\partial\mathbf{y}}{\partial\mathbf{k}}=\left[\frac{\partial\mathbf{y}}{\partial k_1},\cdots,\frac{\partial\mathbf{y}}{\partial k_p}\right]=[\mathbf{g}_1,\cdots,\mathbf{g}_p]
\end{equation}
where $\mathbf{g}_j$ represents $n$-dimensional vector which is the sensitivity coefficients of the state variables with respect to parameter $k_j$. Each of $\mathbf{g}_j$ satisfies the differential equation for Jacobian such that
\begin{equation*}
\frac{\text{d}\mathbf{g}_j(t)}{\text{d}t}=\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{g}_j+\frac{\partial\mathbf{f}}{\partial k_j};\hspace{10mm}\mathbf{g}_j(t_0)=0;\hspace{10mm}j=1,\cdots,p
\end{equation*}
We generate $n\times(p+1)$-dimensional differential equations system
\begin{equation*}
\frac{d\mathbf{z}}{dt}=\varphi(\mathbf{z})
\end{equation*}
$\mathbf{z}$ is $n\times(p+1)$-dimensional vector
\begin{equation*}
\mathbf{z}=\begin{bmatrix} \mathbf{x}(t)\\
                          \frac{\partial\mathbf{y}}{\partial k_1}\\
                          \vdots\\
                          \frac{\partial\mathbf{y}}{\partial k_p}
\end{bmatrix}
=\begin{bmatrix} \mathbf{y}(t)\\
                 \mathbf{g}_1(t)\\
                 \vdots\\
                 \mathbf{g}_p(t)
\end{bmatrix}
\end{equation*}
$\mathbf{\varphi}(\mathbf{z})$ is $n\times(p+1)$-dimensional vector function

\begin{equation*}
\mathbf{\varphi}(\mathbf{z})=\begin{bmatrix}
\mathbf{f}(\mathbf{y},\mathbf{k})\\
\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{g}_1(t)+\frac{\partial\mathbf{f}}{\partial k_1}\\
\vdots\\
\frac{\partial\mathbf{f}}{\partial\mathbf{y}}\mathbf{g}_p(t)+\frac{\partial\mathbf{f}}{\partial k_p}
\end{bmatrix}
\end{equation*}

To get the Jacobian for all $t_i$, $\varphi(\mathbf{z}_i)$ should be solved for $t_i,~~i=1,2,\cdots,N$.


In [ ]:
def phi_breakage(breakage, z, dbs, n, p, delta):
    # dbs: discretized breakage and selection functions
    z = z.astype(np.float)
    y = z[0:n]
    J = z[n:].reshape((p, n)).transpose()
    phiz = np.empty(n * (p+1))
    dfdy = np.empty((n, n))
    dfdk = np.empty((p, n))
    
    for i in range(n):
        yr = y.copy()
        yl = y.copy()
        yr[i] += delta
        yl[i] -= delta
        dfdy[i] = (breakage(yr, dbs[0], dbs[1]) - \
                   breakage(yl, dbs[0], dbs[1])) / (2 * delta)
    dfdy = dfdy.transpose()
    
    for i in range(p):
        dfdk[i] = (breakage(y, dbs[2][i], dbs[3][i]) - \
                   breakage(y, dbs[4][i], dbs[5][i])) / (2 * delta)
    dfdk = dfdk.transpose()
    
    dJdt = dfdy @ J + dfdk
    phiz[0:n] = breakage(y, dbs[0], dbs[1])
    phiz[n:] = dJdt.transpose().flatten()
    return phiz

In [115]:
def dfdytest1(breakage, y, b, S, n, p, delta):
    dfdy = np.empty((n, n))
    
    for i in range(n):
        yr = y.copy()
        yl = y.copy()
        yr[i] += delta
        yl[i] -= delta
        dfdy[i] = (breakage(yr, b, S) - \
                   breakage(yl, b, S)) / (2 * delta)
    dfdy = dfdy.transpose()
    return dfdy

In [119]:
%timeit dfdytest1(breakage, N, b, S, n, p, delta)

1.07 ms ± 3.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [112]:
def dfdytest2(breakage, y, b, S, n, p, delta):
    dfdy = np.empty((n, n))

    for i in range(n):
        yr = y.copy()
        yl = y.copy()
        yr[i] += delta
        yl[i] -= delta
        dfdy[i] = (breakage(yr, b, S) - \
                   breakage(yl, b, S)) / (2 * delta)
    dfdy = np.asarray(dfdy).transpose()
    return dfdy

In [120]:
%timeit dfdytest2(breakage, N, b, S, n, p, delta)

1.4 ms ± 2.79 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [114]:
dfdytest1(breakage, N, b, S, n, p, delta) - dfdytest2(breakage, N, b, S, n, p, delta)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [58]:
Nr = np.repeat(N.reshape(1,n),n,axis=0)
Nl = Nr.copy()

In [59]:
Nr = Nr + delta * np.eye(n)

In [60]:
Nl = Nl - delta * np.eye(n)

In [61]:
Nr - Nl

array([[0.002, 0.   , 0.   , 0.   , 0.   ],
       [0.   , 0.002, 0.   , 0.   , 0.   ],
       [0.   , 0.   , 0.002, 0.   , 0.   ],
       [0.   , 0.   , 0.   , 0.002, 0.   ],
       [0.   , 0.   , 0.   , 0.   , 0.002]])

In [69]:
dfdytest2(breakage, N, b, S, n, p, delta)

array([[2.05618234e-001, 5.02034658e+175, 6.39827228e+170,
        1.90381329e-001, 4.24547314e-001],
       [9.35615439e-067, 1.47763641e+248, 2.40001751e-001,
        7.69165785e+218, 1.35617292e+248],
       [5.72166720e+174, 1.70388495e-051, 1.33473410e+000,
        2.62599145e+179, 3.25660880e-002],
       [6.32299154e+233, 6.48224638e+170, 5.22411352e+257,
        5.74020278e+180, 3.28616953e-001],
       [1.41529402e+161, 6.00736899e-067, 4.31300550e+097,
        1.14482622e-071, 3.58834401e+126]])

In [30]:
np.allclose(dfdytest1(breakage, N, b, S, n, p, delta), dfdytest2(breakage, N, b, S, n, p, delta))

False

In [100]:
[None] *(10,10)

TypeError: can't multiply sequence by non-int of type 'tuple'

In [109]:
a = [[1,2],[3,4]]

In [111]:
a

[[1, 2], [3, 4]]